## TYTANチュートリアル（数字を均等に2組に分ける）

2023年5月20日

ビネクラ安田

出典：[量子アニーリング（QUBO）で複数の数字を均等に2組に分ける」を解く](https://vigne-cla.com/21-25/)

### 問題

**次の6つの自然数を、総和が等しくなるように2グループに分けなさい。**<br>
**15, 25, 33, 41, 64, 82**


<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-25_bmeiqbd.jpg" width = 50%>
</div>

入試問題として出題される場合は倍数、余りなど整数の性質を利用して解けるようになっている。さすがに入試で全探索（力まかせ探索）は出ないと信じる・・・

### QUBOモデルでは何が設定できるか？（おさらい）

基本の条件で、**「4個の量子ビットから2個を1にする」**は
```
H = (q0 + q1 + q2 + q3 - 2)**2
```

これはつまり、4個の量子ビットを足した値が2になるということ。合計値が2であればエネルギーは最小となり、合計値が2からずれるほどエネルギーが高くなる。

ここで、4つの量子ビットを足す際に1, 2, 3, 4の重みをかけてみる。
```
H = (1*q0 + 2*q1 + 3*q2 + 4*q3 - 7)**2
```

これは、**<font color="red">「重さ1, 2, 3, 4のボールのどれを取れば合計7の重さになりますか？」</font>**を解くことに相当している。量子ビットが1になったボールは取る、0になったボールは取らない。取ったボールの合計が7になる。

この方程式的な考え方はチュートリアル26「カクラス」、27「連立方程式を解く」でも使われている。 → [チュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)

今回はこれだけを使うが、その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

まず、すべての数字の合計が260なので各グループは合計130である。

数字に6個の量子ビット（q0～q5）を割り当て、1になったらグループA、0になったらグループBとする。

グループAの量子ビットは1であることを利用し、重みとして数字をかけた合計が130になるような式を設定する。<font color="red">「量子ビットが1になったときにその数字の重みが有効」</font>になる。

```
#グループA（＝１になった量子ビット）は合計130
H = (15*q0 + 25*q1 + 33*q2 + 41*q3 + 64*q4 + 82*q5 - 130)**2
```

反対に、グループBの量子ビットは0であることから、こちらも重みをかけたものの合計が130になるように設定する。ここでテクニックとして、**q0 を (1 – q0) と置き換える**ことで０，１の条件をひっくり返し<font color="red">「0になったときに重みが有効」</font>を表現できる。

```
#グループB（＝０になった量子ビット）は合計130
H = (15*(1-q0) + 25*(1-q1) + 33*(1-q2) + 41*(1-q3) + 64*(1-q4) + 82*(1-q5) - 130)**2
```

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [4]:
from tytan import symbols, Compile, sampler

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')

#グループA（＝１になった量子ビット）は合計130
H = 0
H += (15*q0 + 25*q1 + 33*q2 + 41*q3 + 64*q4 + 82*q5 - 130)**2

#グループB（＝０になった量子ビット）は合計130
H += (15*(1-q0) + 25*(1-q1) + 33*(1-q2) + 41*(1-q3) + 64*(1-q4) + 82*(1-q5) - 130)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#結果
for r in result:
    print(r)

[{'q0': 0.0, 'q1': 1.0, 'q2': 0.0, 'q3': 1.0, 'q4': 1.0, 'q5': 0.0}, -33800.0, 130]
[{'q0': 1.0, 'q1': 0.0, 'q2': 1.0, 'q3': 0.0, 'q4': 0.0, 'q5': 1.0}, -33800.0, 124]
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 1.0, 'q4': 0.0, 'q5': 1.0}, -33702.0, 129]
[{'q0': 1.0, 'q1': 1.0, 'q2': 1.0, 'q3': 0.0, 'q4': 1.0, 'q5': 0.0}, -33702.0, 117]


最適解は2通り出るがグループ名を入れ替えたものなので同じ。

[15, 25, 33, 41, 64, 82] => [0, 1, 0, 1, 1, 0] ということで、グループAに [25, 41, 64]、グループBに [15, 33, 82] と分かった。

さて、気がついたかもしれないが実はグループBの設定式は無くてもOK。グループAの条件を満たしていれば必然的にグループBの条件も満たすから。

グループBの条件式をコメントアウトして実行してみよう。